In [1]:
# For Google Colaboratory
import sys, os
if 'google.colab' in sys.modules:
    # mount google drive
    from google.colab import drive
    drive.mount('/content/gdrive')
    # find automatically the path of the folder containing "file_name" :
    file_name = 'data_test_demo.ipynb'
    import subprocess
    path_to_file = subprocess.check_output('find . -type f -name ' + str(file_name), shell=True).decode("utf-8")
    path_to_file = path_to_file.replace(file_name,"").replace('\n',"")
    # if previous search failed or too long, comment the previous line and simply write down manually the path below :
    print(path_to_file)
    # change current path to the folder containing "file_name"
    os.chdir(path_to_file)
    !pwd
    

In [3]:
import pandas as pd
from sklearn import model_selection
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from random import randint
import time
import utils

In [5]:
df = pd.read_csv("A_Z Handwritten Data.csv")
device = torch.device("cpu")

FileNotFoundError: [Errno 2] No such file or directory: 'A_Z Handwritten Data.csv'

In [ ]:
for i in range(1, 10):
    print(df.iloc[60000 + i, 0])
    plt.subplot(5, 5, i)
    digit = np.array(df.iloc[60000 + i, 1:])
    plt.imshow(digit.reshape(28, 28))

In [ ]:
data_feature, data_label = np.array(df.iloc[:, 1:]), np.array(df.iloc[:, 0])
train_data, test_data, train_label, test_label = model_selection.train_test_split(
    data_feature, data_label, test_size=0.05, random_state=75)
ss = MinMaxScaler()
train_data = ss.fit_transform(train_data)
test_data = ss.fit_transform(test_data)

In [ ]:
train_data = (torch.Tensor(train_data.reshape(-1,28,28))).to(device)
test_data = (torch.Tensor(test_data.reshape(-1,28,28))).to(device)
train_label = (torch.LongTensor(train_label)).to(device)
test_label = (torch.LongTensor(test_label)).to(device)

In [ ]:
class two_layer_net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(two_layer_net, self).__init__()

        self.layer1 = nn.Linear(input_size, hidden_size, bias=False)
        self.layer2 = nn.Linear(hidden_size, output_size, bias=False)

    def forward(self, x):

        y = self.layer1(x)
        y_hat = F.relu(y)
        scores = self.layer2(y_hat)

        return scores

In [ ]:
net = two_layer_net(784, 50, 26)
net = net.to(device)

print(net)
utils.display_num_param(net)

In [ ]:
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(net.parameters(), lr=0.2)

bs = 10000

In [ ]:
def eval_on_test_set():
    bs=100
    running_error = 0
    num_batches = 0

    for i in range(0, 18600, bs):

        minibatch_data = test_data[i:i + bs]
        minibatch_label = test_label[i:i + bs]

        inputs = (minibatch_data).view(bs, 784)

        scores = net(inputs)

        error = utils.get_error(scores, minibatch_label)

        running_error += error.item()

        num_batches += 1

    total_error = running_error / num_batches
    print('test error  = ', total_error * 100, 'percent')

In [ ]:
start = time.time()

for epoch in range(800):

    running_loss = 0
    running_error = 0
    num_batches = 0

    shuffled_indices = torch.randperm(350000)

    for count in range(0, 350000, bs):

        # forward and backward pass

        optimizer.zero_grad()

        indices = shuffled_indices[count:count + bs]
        minibatch_data = train_data[indices]
        minibatch_label = train_label[indices]

        inputs = minibatch_data.view(bs, 784)

        inputs.requires_grad_()

        scores = net(inputs)

        loss = criterion(scores, minibatch_label)

        loss.backward()

        optimizer.step()

        # compute some stats

        running_loss += loss.detach().item()

        error = utils.get_error(scores.detach(), minibatch_label)
        running_error += error.item()

        num_batches += 1

    # once the epoch is finished we divide the "running quantities"
    # by the number of batches

    total_loss = running_loss / num_batches
    total_error = running_error / num_batches
    elapsed_time = time.time() - start

    # every 10 epoch we display the stats
    # and compute the error rate on the test set

    if epoch % 10 == 0:

        print(' ')

        print('epoch=', epoch, '\t time=', elapsed_time, '\t loss=',
              total_loss, '\t error=', total_error * 100, 'percent')

        eval_on_test_set()

In [ ]:
import utils
# choose a picture at random
idx=np.random.randint(0, 18600)
im=test_data[idx]

# diplay the picture
utils.show(im.cpu())

# feed it to the net and display the confidence scores
scores =  net( im.view(1,784)) 
probs= F.softmax(scores, dim=1)
utils.show_prob_alphabets(probs.cpu())
